In [ ]:
#r "nuget: Microsoft.PowerBI.Api, 4.10.0"
#r "nuget: Microsoft.IdentityModel.Clients.ActiveDirectory, 5.2.9"
#r "nuget: Azure.Identity, 1.7.0"
#r "nuget: Microsoft.Rest.ClientRuntime, 2.3.24"
#r "nuget: Microsoft.Azure.Storage.Blob"
#r "nuget: Microsoft.Azure.Storage.Common"
#r "nuget: Azure.Security.KeyVault.Secrets, 4.4.0"
#r "nuget: CsvHelper"
#r "nuget: microsoft.aspnetcore.mvc.core"

In [ ]:
%%csharp
using System;
using System.Linq;
using System.Configuration;
using System.IO;
using System.Security;
using System.Reflection;
using Microsoft.Rest;
using Microsoft.Identity.Client;
using Microsoft.PowerBI.Api;
using System.Collections.Generic;
using Microsoft.Spark.Sql;
using System.Collections.Generic;
using System.Net.Http;
using System.Net.Http.Headers;
using Newtonsoft.Json.Linq;
using System.IO.Compression;
using Microsoft.Azure.Storage.Blob;
using System.Threading;
using System.Threading.Tasks;
using Microsoft.Spark.Sql;
using Microsoft.Spark.Sql.Types;
using Microsoft.Azure.Storage.Auth;
using Microsoft.Azure.Storage;
using Microsoft.Extensions.Logging;
using System.Net;
using Newtonsoft.Json;
using System.Globalization;
using Newtonsoft.Json.Serialization;
using CsvHelper;
using Microsoft.AspNetCore.Mvc;
using Microsoft.PowerBI.Api;

private static string spnAccessToken = string.Empty;

static string GetSPNAccessToken(string applicationId, string applicationSecret, string tenantSpecificAuthority) {
        if (spnAccessToken.Equals(string.Empty)) {
        var appConfidential = ConfidentialClientApplicationBuilder.Create(applicationId)
                                .WithClientSecret(applicationSecret)
                                .WithAuthority(tenantSpecificAuthority)
                                .Build();

        string[] scopesDefault = new string[] { "https://analysis.windows.net/powerbi/api/.default" };
        var authResult = appConfidential.AcquireTokenForClient(scopesDefault).ExecuteAsync().Result;
        spnAccessToken = authResult.AccessToken;
        }
        return spnAccessToken;
    }
    
public static PowerBIClient GetPowerBiAppOnlyClient(string applicationId
                                                    ,string applicationSecret
                                                    ,string tenantSpecificAuthority
                                                    ,string urlPowerBiServiceApiRoot) {
    var tokenCredentials = new TokenCredentials(GetSPNAccessToken(applicationId, applicationSecret, tenantSpecificAuthority), "Bearer");
    return new PowerBIClient(new Uri(urlPowerBiServiceApiRoot), tokenCredentials);
}

In [ ]:
%%pyspark
from pyspark.sql.types import StructType,StructField,StringType,DateType
from pyspark.sql.functions import *
import os
import re
from notebookutils import mssparkutils

keyvaultName = os.getenv("keyvaultName")  # Create a spark.yarn.appMasterEnv.keyvaultName property in Apache Spark Configuration and store the keyvalut name
applicationId = mssparkutils.credentials.getSecret(keyvaultName,"secret name","linked service name") 
applicationSecret = mssparkutils.credentials.getSecret(keyvaultName,"secret name","linked service name")
storageConnString = mssparkutils.credentials.getSecret(keyvaultName,"secret name","linked service name")
tenantId = mssparkutils.credentials.getSecret(keyvaultName,"secret name","linked service name")


paramData = [(applicationId, applicationSecret, storageConnString, tenantId)]
schema = StructType([ \
  StructField("applicationId", StringType(), True), \
  StructField("applicationSecret", StringType(), True), \
  StructField("storageConnString", StringType(), True), \
  StructField("tenantId", StringType(), True) \
])

df = spark.createDataFrame(spark.sparkContext.parallelize(paramData), schema)
df.createOrReplaceTempView("vw_tompo_params")

metadatadf = spark.read.option("header", True).csv("/data/tompo/tompo_report_metadata.csv")
metadatadf.createOrReplaceTempView("vw_report_metadata")

In [ ]:
%%csharp
public class TompoSecret
{
    public string applicationId { get; set; }
    public string applicationSecret { get; set; }
    public string storageConnString { get; set; }
    public string tenantId { get; set; }
}

In [ ]:
%%csharp
using System.Collections.Generic;
using System.IO;
using System.Net.Http;
using System.Net.Http.Headers;
using Newtonsoft.Json.Linq;
using System.IO.Compression;
using Microsoft.Azure.Storage.Blob;
using System.Threading;
using System.Threading.Tasks;
using Microsoft.Spark.Sql;
using Microsoft.Spark.Sql.Types;
using Microsoft.Azure.Storage.Auth;
using Microsoft.Azure.Storage;
using Microsoft.Extensions.Logging;
using System.Net;
using Newtonsoft.Json;
using System.Globalization;
using Newtonsoft.Json.Serialization;
using CsvHelper;
using Microsoft.AspNetCore.Mvc;
using Microsoft.PowerBI.Api;

public const string urlPowerBiServiceApiRoot = "https://api.powerbi.com/";
private static string workspaceId = "";
private static string reportId = "";
private static string reportName = "";
private static string applicationId = "";
private static string applicationSecret = "";
private static string tenantSpecificAuthority = "";
private static string storageConnString = "";


TompoSecret secretObj = new TompoSecret();
var secretsdata = spark.Sql("SELECT * FROM vw_tompo_params");

secretsdata.Collect().ToList().ForEach(row => {
    secretObj.applicationId = row[0].ToString();
    secretObj.applicationSecret = row[1].ToString();
    secretObj.storageConnString = row[2].ToString();
    secretObj.tenantId = row[3].ToString();
    }
);

applicationId = secretObj.applicationId;
applicationSecret = secretObj.applicationSecret;
tenantSpecificAuthority = "https://login.microsoftonline.com/" + secretObj.tenantId;
storageConnString = secretObj.storageConnString;

PowerBIClient pbiClient = GetPowerBiAppOnlyClient(applicationId, applicationSecret, tenantSpecificAuthority, urlPowerBiServiceApiRoot);
var storageAccount = CloudStorageAccount.Parse(storageConnString);
var blobClient = storageAccount.CreateCloudBlobClient();
var container = blobClient.GetContainerReference("hrsisynapsefs");

var metadata = spark.Sql("SELECT * FROM vw_report_metadata where isActive=1");

metadata.Collect().ToList().ForEach(row => {
    var workspaceName = row[0].ToString();
    var workspaceId = row[1].ToString();
    var reportName = row[2].ToString();
    var reportId = row[3].ToString();
    var modelName = row[4].ToString();
    
    var reportStream = pbiClient.Reports.ExportReport(new Guid(workspaceId), new Guid(reportId));
    var blob = container.GetBlockBlobReference("data/tompo/tompo_layout/" + reportName + ".pbix");
    blob.Properties.ContentType = "mutipart/form-data";
    blob.UploadFromStream(reportStream);
    reportStream.Close();
    Console.WriteLine(reportName + " Report has been donwloaded successfully");

    }
);

In [ ]:
%%pyspark
from zipfile import ZipFile
import os
import shutil
import pandas as pd
import json
from os.path import exists

hrsiBasePath = os.getenv("hrsiBasePath")

                         #uncomment below block if running for first time, need to ad exists code
#mssparkutils.fs.unmount("/hrsisynapsefs_temp") 
#mssparkutils.fs.mount(hrsiBasePath, 
#					"/hrsisynapsefs_temp",
#					{"linkedService":"HRBIADLS"}
#					)

synpasefspath = "/synfs/" + mssparkutils.env.getJobId() + "/hrsisynapsefs_temp"

outfilepath = synpasefspath + "/data/tompo/tompo_parseroutput/tompo_output.csv"

file_exists = exists(outfilepath)

if file_exists:
    print("output file exists")
else:
    outfiledf = pd.DataFrame(columns = ['WorkspaceName', 'ReportName', 'PageName', 'VisualType', 'Column', 'ModelName', 'LastRefreshedOn'])
    outfiledf.to_csv(outfilepath, index=False)
    print("created empty one time output file")


metadatadfdata = spark.sql("SELECT * FROM vw_report_metadata where isActive=1")

df = pd.DataFrame(columns = ['WorkspaceName', 'ReportName', 'PageName', 'VisualType', 'Column', 'ModelName', 'LastRefreshedOn'])

for row in metadatadfdata.collect():
    print("Working on file: " + row["ReportName"])
    
    f = ZipFile(synpasefspath + "/data/tompo/tompo_layout/" + row["ReportName"] + ".pbix", 'r')
    f.extractall(synpasefspath + "/data/tompo/tompo_layout/_temp" + row["ReportName"])
    shutil.copyfile(synpasefspath + "/data/tompo/tompo_layout/_temp" + row["ReportName"] + "/Report/Layout", synpasefspath + "/data/tompo/tompo_layout/" + row["ReportName"] + "_Layout")
    shutil.rmtree(synpasefspath + "/data/tompo/tompo_layout/_temp" + row["ReportName"])
    print("Retreived layout file for report " + row["ReportName"] + ".pbix")

    # code from here is to parse layout file and append in csv for POWER BI reporting
    with  open(synpasefspath + "/data/tompo/tompo_layout/" + row[2] + "_Layout", encoding="utf-16le", errors="backslashreplace") as file:
        data = file.read().strip()

    layoutdata = json.loads(data)

    outputlist = []

    for section in layoutdata['sections']:

        tabname = section['displayName']

        for container in section['visualContainers']:
            # if container['id'] != 0 :
            configdict = json.loads(container['config'])

            if 'singleVisual' in configdict:

                visualtype = configdict['singleVisual']['visualType']

                if 'projections' in configdict['singleVisual']:

                    for key in configdict['singleVisual']['projections']:
                        for query in configdict['singleVisual']['projections'][key]:

                            data = query['queryRef'].split('.')[-1]
                            outputlist.append(row["WorkspaceName"])
                            outputlist.append(row["ReportName"])
                            outputlist.append(tabname)
                            outputlist.append(visualtype)
                            outputlist.append(data)
                            outputlist.append(row["ModelName"])
                            outputlist.append(str(pd.to_datetime('now').date()))
                            
                            df.loc[len(df)] = outputlist
                            outputlist.clear()

    outputdf = pd.read_csv(outfilepath, header='infer')

    filterval = row["WorkspaceName"].strip() + row["ReportName"].strip()
    outfiltereddf = outputdf[ (outputdf["WorkspaceName"]+outputdf["ReportName"] != filterval ) ]

    finaldf = pd.concat([outfiltereddf, df])
    
    finaldf.to_csv(outfilepath, index=False)
    df = df.iloc[:0]
    print("Layout data is added to final csv for file: " + row["ReportName"])